<div style="background:#FFFFEE; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFQ619 - Data Analytics for Strategic Decision Makers (2024)</div>

# IFQ619 :: UA2 - Extending Analytics (40%)

**IMPORTANT:** Refer to the instructions in Canvas [UA2 - Assignment 2 - extending analytics] *BEFORE* working on this assignment.

#### REQUIREMENTS ####

1. Complete and run the code cell below to display your name, student number, and assignment option
2. Identify an appropriate question (or questions) to be addressed by your overall data analytics narrative
3. Extend your analysis in assignment 1 with:
    - the analysis of additional unstructured data using the Guardian API (See accessing the Guardian API notebook),
    - the use of one machine learning technique (as used in the class materials), and
    - identification of ethical considerations relevant to the analysis (by drawing on class materials).
4. Ensure that you include documentation of your thinking and decision-making using markdown cells
5. Ensure that you include appropriate visualisations, and that they support the overall narrative
6. Ensure that your insights answer your question/s and are appropriate to your narrative. 
7. Ensure that your insights are consistent with the ethical considerations identified.

**NOTE:** you should not repeat the analysis from assignment 1, but you may need to save dataframes from assignment 1 and reload for use in this assignment. You may also summarise your assignment 1 insights as part of the process of identifying questions for analysis.

#### SUBMISSION ####

1. Create an assignment 2 folder named in the form **UA2-surname-idnumber** and put your notebook and any data files inside this folder. Note, do not put large training data in this folder (reference any training data that you used but keep it outside this folder), only keep small data files and models in this folder with your notebook.
2. When you have everything in the correct folder, reset all cells and restart the kernel, then run the notebook completely, checking that all cells have run without error. If you encounter errors, fix your notebook and re-run the process. It is important that your notebook runs without errors only requiring the files in the folder that you have created.
3. When the notebook is error free, zip the entire folder (you can select download folder in Jupyter).
4. Submit the zipped folder in Canvas


In [1]:
# Complete the following cell with your details and run to produce your personalised header for this assignment

from IPython.display import HTML

# personal details
first_name = "Morgan"
last_name = "Meeuwissen"
student_number = "n12240800"

personal_header = f"<h1>{first_name} {last_name} ({student_number})</h1>"
HTML(personal_header)

---


# Themes that use topic modelling?
* Long term affects of funding? still remain in news or short-lived?
* Funding as a result of news (public image leading funding)
* Funding to be lagged one year from the news? Gov funding cyle?
* Impact on Local Jobs: The Guardian’s coverage of labor market issues and economic policies provides a context for evaluating how programs like Advance Queensland impact local employment. Discussions on job creation and displacement relate directly to the ethical consideration of benefiting local communities through such initiatives.

Word limit: 1500–3000 words. Three-four analytics cycles?

**Outline:**

## Context 
from first assignment and final insights + graph? - fairly distributed -> Overarching goals of Advance Queensland... Public perception and does it make a difference?

Ethics: Bias?

# Question 1
What does the media care about show?  How much of the media attention is for indigenous matters? 

### Data
Data source is biased - Avoid further bias by not over-onstraining intial data


### Analysis

Explain Cleaning of data for TDIDF - lemmy, preproccessing, stop words, bigrams

Explain choice of NMf


### Visualisation

Clusters and Weightings


### Insight
Regional not som important

Visualisation Topics Cumulative clusters bar chart by articles.


# Question 2 


### Data
Data: count articles on the main topic and re-introduce Advance QUeensland data with manual extra flags as 1/0

### Analysis
Make a Dataframe and show stats per column?


### Visualisation
Correlelagram


### Insight
Media attention not valid?



ANalysis:



# Question 3
What impacts funding ?  What impacts media attention from the funding 


### Data


### Analysis


### Visualisation
Regression Test confusion matrix

### Insight

Can a relationship be established? Predict funding from public opinion.


Insight: It doesn't work for a linear regression?





# Closing statement


CAN WE TOPIC MODEL ACROSS MULTIPLE CORPUS? TOPICS FROM THE PROGRAM NAME/PROJECT NAME AND FIND MATCHES IN THE GUARDIAN
https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
rain LDA Model on 100,000 Restaurant Reviews from 2016
Grab Topic distributions for every review using the LDA Model
Use Topic Distributions directly as feature vectors in supervised classification models (Logistic Regression, SVC, etc) and get F1-score.
Use the same 2016 LDA model to get topic distributions from 2017 (the LDA model did not see this data!)
Want a topic that combines regional and indigenous? bigram traditional owner

# Past assignment Andrew notes
d your assignment 1 analysis by including consideration of ethical and human factors, including 1 extended analysis technique, and using at least 1 additional data source. Possible additional data sources:

    Guardian API (See accessing the Guardian API notebook)
    Hospital location data (Provided, see data folder)

**NOTE: you should not repeat the analysis from assignment 1, but you may need to save dataframes from assignment 1 and reload for use in this assignment. You may also summarise your assignment 1 insights as part of the process of identifying questions for analysis.**

# Insights
Start off by working out what the QLD gov articles actually show. This will be done through topic modelling variants and then clustering
articles json only has the title as a key and the text, but the title has the date that can be extracted for potential use.


clustering to topics already done lda and nmf?  TFIDF is doc specific, others take into account the whole corpus
Establish topics that match Advance Queensland and then use as predictors by year?

No mentions of Advance QLD
QLD Gov search not finding a cluster for Regional
**Try working from regional queensland back? Just regional? If taking a specific approach like this could have way less clusters....**

https://open-platform.theguardian.com/explore/ very useful. Narrow down from 79

In [2]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pandas as pd
import json

import re
import spacy
import plotly.express as px

In [3]:
# # Load the data - articles from The Guardian about the war in Ukraine
# file_path = "data/"
# file_name = "qld_articles.json"

# with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
#     articles = json.load(fp)

# #print(f"Loaded {len(articles)} articles from {file_name}")


# # AYOOOOOO WE DOING A LEMMATISING FUNCTION
# # https://jonathansoma.com/lede/image-and-sound/text-analysis/text-analysis-word-counting-lemmatizing-and-tf-idf/


# nlp = spacy.load("en_core_web_sm")
# # Source: spaCy is a dream, but a dream where sometimes your legs won’t move right and you can’t read text. But sometimes you can fly! So yes, as always, ups and downs
# # Morgan: I hate this
# # python -m spacy download en
# # python -m pip install spacy

# def lemmatize(text):
#     doc = nlp(text)
#     # Turn it into tokens, ignoring the punctuation
#     tokens = [token for token in doc if not token.is_punct]
#     # Convert those tokens into lemmas, EXCEPT the pronouns, we'll keep those.
#     lemmas = [token.lemma_ if token.pos_ != 'PRON' else token.orth_ for token in tokens]
#     return lemmas


# # Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# # Count a maximum of 10000 terms, and remove common english stop words
# StopWords = list(ENGLISH_STOP_WORDS.union(["Monday","Tuesday","Wednesday","Thursday","Friday", "Saturday", "Sunday","nbsp", "\n", "|", "\n ", 
#                                            "$", "year", "m", "new", "need", "increase","bst", "gmt", "says", "year", "told","day","know", "We", 
#                                            "'ve","'re", "read", "today", "day","like","I", "'ve","I've","I'm","just","use", "think", "story"]))


In [4]:
# # TFIDF
# # Only count terms that in maximum of 75% of documents, and a minimum of 5 documents. 
# # Count a maximum of 10000 terms, and remove common english stop words
# tfidf_vectorizer = TfidfVectorizer(
#                                    #reprocessor=lambda x: re.sub(r"([\d\.])+", "NUM", x),
#                                    max_df=0.75,min_df=5,max_features=10000,
#                                    stop_words=StopWords, #Add stop words
#                                    #tokenizer=lemmatize, # Lemmatiseer
#                                    ngram_range = (1,2) #Use Bigrams as well to pick up things like "First Nation"
# )


# tfidf_dt_matrix = tfidf_vectorizer.fit_transform(articles.values())

Introduce NMF

In [5]:
# # Set number of topics
# num_topics = 20
# # Set max number of iteractions
# max_iterations = 2500
# feature_names = tfidf_vectorizer.get_feature_names_out()

# # Create the model
# nmf_model = NMF(n_components=num_topics,init='random', random_state=37  # Set random state to have reproducible results
#                 ,beta_loss='frobenius', max_iter=max_iterations)

# # Fit the model to the data and use it to transform the data
# doc_topic_nmf = nmf_model.fit_transform(tfidf_dt_matrix)

# topic_term_nmf = nmf_model.components_

In [6]:
# # Get the topics and their terms
# nmf_topic_dict = {}
# for index, topic in enumerate(topic_term_nmf):
#     zipped = zip(feature_names, topic)
#     top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
#     #print(top_terms)
#     top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
#     nmf_topic_dict[f"nmf{index}"] = top_terms_list

# nmf_terms_df = []
# # Print the topics with their terms    
# for k,v in nmf_topic_dict.items():
#     #print(k)
#     #print(v)
#     #print()
#     for t, w in v.items():
#         nmf_terms_df.append([k, t, w])

# nmf_terms_df = pd.DataFrame(nmf_terms_df)
# nmf_terms_df.rename(columns={nmf_terms_df.columns[0]: "Topic Cluster",
#                                        nmf_terms_df.columns[1]: "Term",
#                                        nmf_terms_df.columns[2]: "Weight" }, inplace=True)

# topic_map_dict = {"nmf0":"Politics 1", 
#                   "nmf1":"Youth Crime",
#                   "nmf2":"Family",
#                   "nmf3":"Politics 2",
#                   "nmf4":"Environment",
#                   "nmf5":"COVID 1",
#                   "nmf6":"Aged Care",
#                   "nmf7":"Mining",
#                   "nmf8":"Power",
#                   "nmf9":"Police",
#                   "nmf10":"Great Barrier Reef",
#                   "nmf11":"Flood",
#                   "nmf12":"Education",
#                   "nmf13":"Domestic Violence",
#                   "nmf14":"Sport",
#                   "nmf15":"Cost of Living",
#                   "nmf16":"Other",
#                   "nmf17":"Climate Change",
#                   "nmf18":"First Nation",
#                   "nmf19":"COVID 2"}

# # Write the Terms DF to a csv for later use
# nmf_terms_df = nmf_terms_df.replace({"Topic Cluster": topic_map_dict})
# nmf_terms_df.to_csv("data/nmf_terms_df.csv")

# # Write the topics DF to a csv for later use
# doc_topic_nmf_df =  pd.DataFrame(doc_topic_nmf,index=articles.keys(), columns= nmf_model.get_feature_names_out())
# doc_topic_nmf_df = doc_topic_nmf_df.rename(columns = topic_map_dict)
# doc_topic_nmf_df.reset_index(inplace= True)
# doc_topic_nmf_df.to_csv("data/nmf_topics_df.csv", index = False)


In [7]:
nmf_terms_df = pd.read_csv("data/nmf_terms_df.csv")
nmf_topics_df = pd.read_csv("data/nmf_topics_df.csv")

### **Topics:**

Resources
Indigenous 
Adani
Youth Crime
COVID
Energy
Great Barrier Reef
Adani Environmental Impact
Environment
Housing
Domestic Violence
Other 1
Other 2
Flooding
Sport


In [8]:
# Your answer here




nmf_topic_terms_fig = px.bar(nmf_terms_df,
       x ="Weight",
       y = "Term",
       facet_col="Topic Cluster", 
       facet_col_wrap=5,
       orientation='h',
       title = "Total Contractual Commitment ($ GST excl.) by Local Government Area",
       labels = {"Local Government Area (LGA)": "Local Government Area", 
                 "Total Contractual Commitment ($ GST excl.)": "Contractual Commitment ($ GST excl.)"})

nmf_topic_terms_fig.update_layout(
    title_font_size=25,
    title_x=0.5,
    legend_title_font_size=15,
    height=1000
)
nmf_topic_terms_fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])) # https://plotly.com/python/facet-plots/
nmf_topic_terms_fig.update_yaxes(showticklabels=True, matches=None)



In [9]:
nmf_topics_df

,index,Politics 1,Youth Crime,Family,Politics 2,Environment,COVID 1,Aged Care,Mining,Power,...,Great Barrier Reef,Flood,Education,Domestic Violence,Sport,Cost of Living,Other,Climate Change,First Nation,COVID 2
0,Queensland premier wedged as casino operator f...,0.000000,0.023006,0.025470,0.057121,0.000000,0.004943,0.000000,0.011313,0.003915,...,0.000045,0.000000,0.000000,0.000000,0.011754,0.048578,0.006951,0.000000,0.000000,0.000000
1,Baby scalded with hot liquid in Brisbane park ...,0.000000,0.008911,0.075376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Man dead after accident at central Queensland ...,0.000000,0.000000,0.013692,0.000000,0.000000,0.000000,0.006315,0.010302,0.004265,...,0.000000,0.003125,0.000000,0.000000,0.000000,0.000000,0.000000,0.001171,0.000000,0.000126
3,‘It’s okay to poo at work’: Australian health ...,0.000000,0.014390,0.050645,0.003509,0.001530,0.028427,0.012827,0.000000,0.000000,...,0.000577,0.000000,0.006822,0.001656,0.000000,0.006619,0.011636,0.001236,0.006878,0.015117
4,Queensland budget 2024: the biggest winners an...,0.002218,0.067343,0.000000,0.031064,0.002112,0.008095,0.000000,0.005385,0.032732,...,0.000002,0.003022,0.001812,0.000000,0.004821,0.097726,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11861,'We can’t lock young people up': Victoria warn...,0.000000,0.409210,0.001695,0.000000,0.000000,0.005263,0.004330,0.000000,0.001905,...,0.000000,0.003242,0.000000,0.001612,0.002039,0.001831,0.000000,0.002949,0.000000,0.000000
11862,'He speaks for us': why Tamworth is oozing sym...,0.092632,0.000000,0.082300,0.060249,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.001259,0.000000,0.000000,0.001776,0.000000,0.000000
11863,Cents and sensibility: the cost and meaning of...,0.000000,0.032321,0.104888,0.000000,0.003003,0.000000,0.022768,0.000000,0.004558,...,0.002086,0.008670,0.000000,0.000000,0.000000,0.044395,0.000000,0.000000,0.008301,0.000000
11864,The full list of Trump's 'under-reported' terr...,0.000000,0.000000,0.006297,0.000000,0.000000,0.000000,0.000000,0.000000,0.001953,...,0.003292,0.000000,0.000000,0.000000,0.000000,0.000000,0.027074,0.016659,0.000000,0.000000


In [10]:
# Below gets the topics 
nmf_topics_df = nmf_topics_df.rename(columns={"index":"article"})

nmf_topics_df["article_year"] = nmf_topics_df["article"].str.extract(r"(?<=\[)(\d{4})(?=\-)")

nmf_topics_df = nmf_topics_df.drop('article', axis=1)


nmf_topics_df.set_index("article_year", inplace= True)
#Normalise the weights
nmf_topics_df = nmf_topics_df.divide(nmf_topics_df.sum(axis =1), axis = 0)

sum_topic_nmf_df  = nmf_topics_df.sum().reset_index().rename(columns={"index":"Topic", "0":"Total NMF Topic Weight Across all Articles(Should normalise this really)"})
sum_topic_nmf_df = sum_topic_nmf_df.rename(columns={"index":"Topic", 0:"Total NMF Topic Weight Across all Articles(Should normalise this really)"})

px.bar(sum_topic_nmf_df, x = "Topic", y = "Total NMF Topic Weight Across all Articles(Should normalise this really)")